In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip -q install torch torchvision pandas pillow tqdm --progress-bar off

In [3]:
import os, torch, pandas as pd, numpy as np
from PIL import Image
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torchvision.models as models
import torch.nn as nn


In [8]:
IMAGES_DIR = "/content/drive/MyDrive/Farzana_Mimic/afib_sample_images"
META_CSV = "/content/drive/MyDrive/Farzana_Mimic/icd_matched_with_cxr.csv"# has study_id, dicom_id, (+ other cols)
OUT_CSV    = "/content/drive/MyDrive/Farzana_Mimic/cxr_embeddings_by_dicom.csv"

#metadata to keep in the output
KEEP_META = ["subject_id", "study_id", "dicom_id", "hadm_id", "seq_num", "icd_code", "icd_version"]

meta = pd.read_csv(META_CSV)
meta = pd.read_csv(META_CSV)

In [5]:
meta.shape


(51844, 7)

In [11]:
files = os.listdir(IMAGES_DIR)
len(files)

300

# **Farzana's Sample Code**

In [ ]:
#filename = dicom_id + ".jpg"
def find_path_for_dicom(dicom_id):
    #exact name
    candidates = [
        os.path.join(IMAGES_DIR, f"{dicom_id}.jpg"),
        os.path.join(IMAGES_DIR, f"{dicom_id}.jpeg"),
        os.path.join(IMAGES_DIR, f"{dicom_id}.png"),
    ]
    for p in candidates:
        if os.path.exists(p):
            return p
    for f in os.listdir(IMAGES_DIR):
        if f.lower().startswith(str(dicom_id).lower()):
            return os.path.join(IMAGES_DIR, f)
    return None

meta["filepath"] = meta["dicom_id"].apply(find_path_for_dicom)
meta = meta.dropna(subset=["filepath"]).reset_index(drop=True)

#transforms
transform = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

class CXRDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True); self.t = transform
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        img = Image.open(self.df.loc[i, "filepath"]).convert("RGB")
        return (self.t(img) if self.t else img), i

loader = DataLoader(CXRDataset(meta, transform), batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

#DenseNet121 feature extractor (1024-D)
device = "cuda" if torch.cuda.is_available() else "cpu"
backbone = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
feature_extractor = nn.Sequential(
    backbone.features,
    nn.ReLU(inplace=True),
    nn.AdaptiveAvgPool2d((1,1)),
    nn.Flatten(),
).to(device).eval()

#final inferenece
emb_list = [None] * len(meta)
with torch.no_grad():
    for x, idxs in tqdm(loader, desc="Embedding CXR (per dicom_id)"):
        x = x.to(device)
        f = feature_extractor(x)                         # (B,1024)
        f = torch.nn.functional.normalize(f, p=2, dim=1)
        for j, irow in enumerate(idxs.tolist()):
            emb_list[irow] = f[j].cpu().numpy().tolist()

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


100%|██████████| 30.8M/30.8M [00:00<00:00, 203MB/s]
Embedding CXR (per dicom_id): 100%|██████████| 1/1 [00:15<00:00, 15.78s/it]


In [ ]:
out = meta[[c for c in KEEP_META if c in meta.columns]].copy()
out["img_embed"] = emb_list
out.to_csv(OUT_CSV, index=False)
print(f"Saved → {OUT_CSV}")

Saved → /content/drive/MyDrive/MIMIC_IV_Pipeline/cxr_embeddings_by_dicom.csv


In [ ]:
#aggregated the embed based on study_id
import ast, numpy as np
df = pd.read_csv(OUT_CSV)
df["img_embed"] = df["img_embed"].apply(ast.literal_eval)
agg = (df.groupby("study_id")["img_embed"]
         .apply(lambda rows: np.mean(np.vstack(rows), axis=0).tolist())
         .reset_index())
agg.to_csv(OUT_CSV.replace(".csv", "_per_study.csv"), index=False)

In [ ]:
import ast, numpy as np
df = pd.read_csv(OUT_CSV)
df

,subject_id,study_id,dicom_id,hadm_id,seq_num,icd_code,icd_version,img_embed
0,10313763,51527697,eee23a20-30e4d811-4ddcd3f7-8637280e-b6a719d7,25494448,1,486,9,"[2.2592575987800956e-06, 5.175691330805421e-05..."
1,10332722,50490989,369581f3-5807e20a-8cf319b5-6d33a899-8432e5d4,26361501,1,3842,9,"[2.869054242182756e-06, 0.00010237601964036003..."
2,10404109,52328790,acf6d62b-f2ee9d0e-d53c8313-f123dfba-a07a1368,29867820,1,51881,9,"[2.9136190278222784e-06, 0.0001181846964755095..."
3,10788120,55149545,3e9af386-d1f758dc-565fc916-cd19b9d5-ceb4829f,22275151,1,1977,9,"[7.315040420508012e-06, 6.534462590934709e-05,..."
4,10800546,51078071,78966895-c3975082-38df2779-81572397-bce1358f,20149451,1,389,9,"[5.187019723962294e-06, 0.00016804327606223524..."
5,11533462,55839159,5c288470-1255240b-c275b4d1-50cc04f4-b33abbce,27328168,1,43491,9,"[4.541793714452069e-06, 0.00015447079204022884..."
6,12133670,51759637,01c47909-29055dd7-7ccfb97f-a38194de-11a8c26d,24487398,1,431,9,"[3.84774921258213e-06, 0.0001566458522574976, ..."
7,12203013,54298796,4a7d6541-9379308d-eb1690bb-0591a394-c18cb362,26782475,1,99932,9,"[4.314239504310535e-06, 0.00017723247583489865..."
8,12827336,56965256,43bff1b4-e5429042-f4eb61e6-f2c6a264-0eeb449d,22910608,1,4329,9,"[5.863133992534131e-06, 0.00018043318414129317..."
9,13239393,50866888,900f01fc-5a91448f-45902b69-7da1019a-a12de045,21027372,1,44024,9,"[3.830833065876504e-06, 6.16093966527842e-05, ..."


# **Fahmida's MultiModel Embedding Code**

In [12]:
import os
import pandas as pd
import torch
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.models as models
import torch.nn as nn
from tqdm import tqdm

In [14]:
# ---------------------------------------------------------
# STEP 1 — Build metadata from your AFIB image folder
# ---------------------------------------------------------
IMAGES_DIR = "/content/drive/MyDrive/Farzana_Mimic/afib_sample_images"

all_files = os.listdir(IMAGES_DIR)
img_files = [f for f in all_files if f.lower().endswith(('.jpg','.jpeg','.png'))]

dicom_ids = [f.rsplit('.', 1)[0] for f in img_files]

meta = pd.DataFrame({"dicom_id": dicom_ids})

def find_correct_path(dicom_id):
    for ext in ["jpg", "jpeg", "png"]:
        p = os.path.join(IMAGES_DIR, f"{dicom_id}.{ext}")
        if os.path.exists(p):
            return p
    return None

meta["filepath"] = meta["dicom_id"].apply(find_correct_path)
meta = meta.dropna(subset=["filepath"]).reset_index(drop=True)

print(f"Total valid images found: {len(meta)}")

Total valid images found: 300


In [15]:
# ---------------------------------------------------------
# STEP 2 — Torch transforms
# ---------------------------------------------------------
transform = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])


In [16]:
# ---------------------------------------------------------
# STEP 3 — Dataset + DataLoader
# ---------------------------------------------------------
class CXRDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.t = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
        img = Image.open(self.df.loc[i, "filepath"]).convert("RGB")
        return (self.t(img) if self.t else img), i

loader = DataLoader(
    CXRDataset(meta, transform),
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)


In [17]:
# ---------------------------------------------------------
# STEP 4 — Define model backbones and extract features
# ---------------------------------------------------------

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

def build_feature_extractor(model_name):
    """Returns a feature extractor (no classification head) and output dimension."""

    if model_name == "densenet121":
        backbone = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
        extractor = nn.Sequential(
            backbone.features,
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten()
        )
        dim = 1024

    elif model_name == "resnet101":
        backbone = models.resnet101(weights=models.ResNet101_Weights.IMAGENET1K_V1)
        extractor = nn.Sequential(
            *(list(backbone.children())[:-1]),
            nn.Flatten()
        )
        dim = 2048

    elif model_name == "efficientnet_b0":
        backbone = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
        extractor = nn.Sequential(
            backbone.features,
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten()
        )
        dim = 1280

    else:
        raise ValueError(f"Unknown model: {model_name}")

    return extractor.to(device).eval(), dim


Using device: cuda


In [18]:
# ---------------------------------------------------------
# STEP 5 — Loop through models and generate embeddings
# ---------------------------------------------------------

MODELS = ["densenet121", "resnet101", "efficientnet_b0"]

OUTPUT_DIR = "/content/drive/MyDrive/Farzana_Mimic/embeddings/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

for model_name in MODELS:
    print(f"\n🔹 Running embeddings for: {model_name}")

    feature_extractor, dim = build_feature_extractor(model_name)

    emb_list = [None] * len(meta)

    with torch.no_grad():
        for x, idxs in tqdm(loader, desc=f"Embedding: {model_name}"):
            x = x.to(device)
            f = feature_extractor(x)                     # (B, dim)
            f = torch.nn.functional.normalize(f, p=2, dim=1)

            for j, irow in enumerate(idxs.tolist()):
                emb_list[irow] = f[j].cpu().numpy().tolist()

    # Save output CSV
    out_df = meta.copy()
    out_df["embedding"] = emb_list

    OUT_FILE = os.path.join(OUTPUT_DIR, f"cxr_embeddings_{model_name}.csv")
    out_df.to_csv(OUT_FILE, index=False)

    print(f"✔ Saved embeddings → {OUT_FILE}")


🔹 Running embeddings for: densenet121
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


100%|██████████| 30.8M/30.8M [00:00<00:00, 83.0MB/s]
Embedding: densenet121: 100%|██████████| 10/10 [01:41<00:00, 10.13s/it]


✔ Saved embeddings → /content/drive/MyDrive/Farzana_Mimic/embeddings/cxr_embeddings_densenet121.csv

🔹 Running embeddings for: resnet101
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


100%|██████████| 171M/171M [00:00<00:00, 183MB/s]
Embedding: resnet101: 100%|██████████| 10/10 [00:23<00:00,  2.30s/it]


✔ Saved embeddings → /content/drive/MyDrive/Farzana_Mimic/embeddings/cxr_embeddings_resnet101.csv

🔹 Running embeddings for: efficientnet_b0
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 167MB/s]
Embedding: efficientnet_b0: 100%|██████████| 10/10 [00:22<00:00,  2.21s/it]


✔ Saved embeddings → /content/drive/MyDrive/Farzana_Mimic/embeddings/cxr_embeddings_efficientnet_b0.csv
